In [1]:
import pandas as pd
import time, datetime
import multiprocessing as mp

- function to produce a list of file URLs

In [22]:
def get_file_list(start, end):
    all_files = []
    url_1 = "http://data.gdeltproject.org/gdeltv2/" # first part of gdelt data link
    url_3 = ".gkg.csv.zip" # file type
    current_time = pd.to_datetime(int(end), format='%Y%m%d%H%M%S')
    date_time = pd.to_datetime(int(start), format='%Y%m%d%H%M%S')

    while date_time < current_time:
        url_2 = date_time.strftime("%Y%m%d%H%M%S")
        url = url_1 + url_2 + url_3 # combine URL parts
        all_files.append(url)
        date_time += datetime.timedelta(minutes = 15) # test adding 15 mins to timestamp
        
    return all_files

- function to read in all files produced by 'get_file_list' function

In [30]:
def load_all_files(file_list):
    # blank list for files
    list_df = []
    
        # loop through all file names and load
    for filename in file_list:
        try:
            df = pd.read_csv(filename, sep="\t", header=None, on_bad_lines="warn", encoding= 'unicode_escape')
            list_df.append(df)
        except UnicodeDecodeError:
            df = pd.read_csv(filename, sep="\t", header=None, on_bad_lines="warn", encoding = "ISO-8859-1")
            list_df.append(df)
        except UnicodeDecodeError:
            df = pd.read_csv(filename, sep="\t", header=None, on_bad_lines="warn")
            list_df.append(df)
        
    
    # combine list of dataframes
    frame = pd.concat(list_df, axis=0, ignore_index=True)
    
    return frame

In [43]:
def load_all_files2(filename):
    
    try:
        df = pd.read_csv(filename, sep="\t", header=None, on_bad_lines="warn", encoding= 'unicode_escape')
        
    except UnicodeDecodeError:
        df = pd.read_csv(filename, sep="\t", header=None, on_bad_lines="warn", encoding = "ISO-8859-1")

    except UnicodeDecodeError:
        df = pd.read_csv(filename, sep="\t", header=None, on_bad_lines="warn")

    return df

In [49]:
all_files = get_file_list("20210101000000", "20210101003000")
all_files

['http://data.gdeltproject.org/gdeltv2/20210101000000.gkg.csv.zip',
 'http://data.gdeltproject.org/gdeltv2/20210101001500.gkg.csv.zip']

- load data and produce data frame without parallel processing

In [33]:
df = []
tic = time.perf_counter()
######
df = load_all_files(all_files)
#######
toc = time.perf_counter()
print(f"Downloaded and appended in {(toc - tic)/60:0.2f} minutes")

Downloaded and appended in 0.09 minutes


In [ ]:
# list_df[-1]
# #20210126221500

In [34]:
df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,20210101000000-0,20210101000000,1,10news.com,https://www.10news.com/news/national/coronavir...,NaN,NaN,TAX_ETHNICITY;TAX_ETHNICITY_CANADIAN;GENERAL_G...,"GENERAL_HEALTH,307;MEDICAL,307;GENERAL_GOVERNM...","1#Canada#CA#CA#60#-96#CA;4#Toronto, Ontario, C...",...,"wc:191,c12.1:10,c12.10:20,c12.12:6,c12.13:3,c1...",https://ewscripps.brightspotcdn.com/80/6b/c73c...,NaN,NaN,https://youtube.com/user/10newsvideos;,NaN,"Affairs Minister Dominic LeBlanc,208;Bill Blai...",NaN,NaN,<PAGE_LINKS>https://apnews.com/article/dominic...
1,20210101000000-1,20210101000000,1,ketchikandailynews.com,https://www.ketchikandailynews.com/news/local/...,"KILL#2#new virusrelated#2#Alaska, United State...","KILL#2#new virusrelated#2#Alaska, United State...",CRISISLEX_CRISISLEXREC;CRISISLEX_O02_RESPONSEA...,"KILL,2569;KILL,2595;KILL,2809;KILL,2888;KILL,4...","3#Eagle River, Alaska, United States#US#USAK#6...",...,"wc:775,c12.1:26,c12.10:32,c12.12:6,c12.13:16,c...",https://bloximages.newyork1.vip.townnews.com/k...,NaN,NaN,NaN,NaN,"Emergency Operations,124;Alaska Department,175...","7,cases on Wednesday,54;6,cases involved Ketch...",NaN,<PAGE_PRECISEPUBTIMESTAMP>20201231185400</PAGE...
2,20210101000000-2,20210101000000,1,grandforksherald.com,https://www.grandforksherald.com/news/accident...,NaN,NaN,TAX_FNCACT;TAX_FNCACT_CHILDREN;TAX_FNCACT_MAN;...,"TAX_FNCACT_DEPUTY,596;TAX_FNCACT_SPOKESPERSON,...","2#Minnesota, United States#US#USMN#45.7326#-93...",...,"wc:262,c12.1:15,c12.10:17,c12.12:8,c12.13:4,c1...",https://www.duluthnewstribune.com/incoming/474...,NaN,NaN,NaN,NaN,"Hill City,539;Aitkin County Sheriff,597;Hill C...","9,children,85;700,Airport Road,390;3,dies,709;...",NaN,<PAGE_LINKS>https://www.duluthnewstribune.com/...
3,20210101000000-3,20210101000000,1,montrealgazette.com,https://montrealgazette.com/news/local-news/pr...,NaN,NaN,GENERAL_HEALTH;MEDICAL;TAX_FNCACT;TAX_FNCACT_M...,"GENERAL_HEALTH,24;GENERAL_HEALTH,484;GENERAL_H...","4#Quebec, Quebec, Canada#CA#CA10#47.5#-72#-571850",...,"wc:196,c12.1:9,c12.10:17,c12.12:3,c12.13:6,c12...",https://smartcdn.prod.postmedia.digital/montre...,NaN,NaN,https://youtube.com/user/themontrealgazette;ht...,NaN,"Minister Christian Dub,47","2,doses,266;2,shots have been administered,311...",NaN,<PAGE_LINKS>https://montrealgazette.com/news/l...
4,20210101000000-4,20210101000000,1,energeticcity.ca,https://energeticcity.ca/2020/12/31/elderly-co...,NaN,NaN,AFFECT;POVERTY;CRISISLEX_C05_NEED_OF_SHELTERS;...,"TAX_WORLDMAMMALS_CAT,578;WB_1609_FOOD_AND_IN_K...",NaN,...,"wc:99,c12.1:3,c12.10:4,c12.12:1,c12.13:1,c12.1...",https://media.socastsrm.com/wordpress/wp-conte...,NaN,NaN,https://youtube.com/channel/UCj6EKBe990fanr4MS...,NaN,"Red Creek Road,74;Annette Alexander,191",NaN,NaN,<PAGE_AUTHORS>Scott Brooks</PAGE_AUTHORS><PAGE...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9583,20210101014500-1167,20210101014500,1,wjr.com,https://www.wjr.com/news/pence-wants-judge-to-...,NaN,NaN,LEADER;TAX_FNCACT;TAX_FNCACT_PRESIDENT;USPEC_P...,"EPU_POLICY_GOVERNMENT,659;TAX_FNCACT_CONGRESSM...","2#Colorado, United States#US#USCO#39.0646#-105...",...,"wc:516,c1.4:1,c12.1:38,c12.10:50,c12.12:14,c12...",NaN,NaN,NaN,NaN,NaN,"Vice President Mike Pence,26;Donald Trump,199;...","1000000,of people,2131;",NaN,<PAGE_TITLE>Pence Wants Judge to Judge to Toss...
9584,20210101014500-1168,20210101014500,1,cbs8.com,https://www.cbs8.com/article/news/year-in-revi...,NaN,NaN,TAX_FNCACT;TAX_FNCACT_ENTERTAINERS;MEDIA_MSM;,"TAX_FNCACT_ENTERTAINERS,12;MEDIA_MSM,61;",NaN,...,"wc:25,c12.1:3,c12.10:5,c12.12:3,c12.13:2,c12.3...",https://media.cbs8.com/assets/CCT/images/63fb3...,NaN,NaN,https://youtube.com/channel/UCMscWako8oDvpdF4p...,NaN,NaN,NaN,NaN,<PAGE_ALTURL_AMP>https://www.cbs8.com/amp/arti...
9585,20210101014500-1169,20210101014500,1,aninews.in,https://www.aninews.in/news/national/general-n...,NaN,NaN,WB_135_TRANSPORT;WB_1801_ROAD_TRANSPORT;WB_164...,"WB_1801_ROAD_TRA

In [ ]:
#df1.to_csv('../00-data/raw_data/2021/raw_jan2021.csv')

- check core count

In [35]:
core_count = mp.cpu_count()
print("Number of processors: ", core_count)

Number of processors:  4


- load data and produce data frame with parallel processing approach 1

In [39]:
tic = time.perf_counter()
#######

with mp.Pool(processes=core_count) as pool:
    df2 = pool.map(load_all_files2, all_files)
    
#######
toc = time.perf_counter()
print(f"Downloaded and appended in {(toc - tic)/60:0.2f} minutes")

In [ ]:
# # combine list of dataframes
# frame = pd.concat(list_df, axis=0, ignore_index=True)
df2

- load data and produce data frame with parallel processing approach 2

In [50]:
tic = time.perf_counter()
#######

pool = mp.Pool(core_count)

df3 = pool.map(load_all_files2, all_files) # data is the list

pool.close()
    
#######
toc = time.perf_counter()
print(f"Downloaded and appended in {(toc - tic)/60:0.2f} minutes")

KeyboardInterrupt: 

In [ ]:
df3

In [62]:
import numpy as np

# Prepare data
np.random.RandomState(100)
arr = np.random.randint(0, 10, size=[2, 5])
print(arr)

[[0 2 8 0 1]
 [6 5 0 7 5]]


In [64]:
data = arr.tolist()
data

[[0, 2, 8, 0, 1], [6, 5, 0, 7, 5]]

In [67]:
# list comprehension
[row for row in data] 

[[0, 2, 8, 0, 1], [6, 5, 0, 7, 5]]

In [68]:
# Redefine, with only 1 mandatory argument.
def howmany_within_range_rowonly(row, minimum=4, maximum=8):
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return count

pool = mp.Pool(mp.cpu_count())

results = pool.map(howmany_within_range_rowonly, data)

pool.close()

print(results)

KeyboardInterrupt: 